# ipyfastscape

## Generate some elevation data from a landscape evolution model (fastscape)

In [ ]:
import numpy as np
import xsimlab as xs
import xarray as xr

from fastscape.models import sediment_model
from ipyfastscape import TopoViz3d

In [ ]:
from fastscape.processes import Escarpment


model = (sediment_model
         .drop_processes('uplift')
         .update_processes({'init_topography': Escarpment}))

In [ ]:
in_ds = xs.create_setup(
    model=model,
    clocks={
        'time': np.arange(0, 4e5 + 2e3, 2e3),
        'out': np.arange(0, 4e5 + 4e3, 4e3),
    },
    master_clock='time',
    input_vars={
        'grid__shape': [101, 201],
        'grid__length': [1e4, 2e4],
        'boundary__status': ['fixed_value', 'core', 'looped', 'looped'],
        'init_topography': {
            'x_left': 1e4,
            'x_right': 1e4,
            'elevation_left': 0.,
            'elevation_right': 1e3
        },
        'flow__slope_exp': 1.,
        'spl': {
            'k_coef_bedrock': 1e-4,
            'k_coef_soil': 1.5e-4,
            'g_coef_bedrock': 1.,
            'g_coef_soil': 1.,
            'area_exp': 0.4,
            'slope_exp': 1.
        },
        'diffusion': {
            'diffusivity_bedrock': 1e-2,
            'diffusivity_soil': 1.5e-2
        }
    },
    output_vars={
        'topography__elevation': 'out',
        'erosion__rate': 'out',
        'drainage__area': 'out',
    }
)

In [ ]:
with xs.monitoring.ProgressBar():
    out_ds = in_ds.xsimlab.run(model=model)

In [ ]:
out_ds

## Terrain 3D visualization

In [ ]:
app = TopoViz3d(out_ds, canvas_height=550, time_dim="out")

app.components['coloring'].set_color_var('erosion__rate')
app.components['coloring'].set_colormap('RdYlGn')
app.components['coloring'].set_color_limits(-2e-3, 2e-3)
app.components['vertical_exaggeration'].set_factor(3)
app.components['background_color'].set_color('#333333')

app.show()

In [ ]:
app.components['coloring'].set_color_var('drainage__area')
app.components['coloring'].set_colormap('Blues')
app.components['coloring'].set_color_scale(log=True)